In [1]:
# ================================
# 1. 라이브러리 & 시드 설정
# ================================
import os
import random
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from tqdm import tqdm
import copy

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)


# ================================
# 2. Dataset 정의 (폴더 기반 이진 분류로 변경)
# ================================
class BinaryImageDataset(Dataset):
    """새로운 폴더 구조 (happy/other)에 맞춰 이미지를 로드합니다."""
    def __init__(self, data_root, split="train", transform=None):
        """
        data_root: /workspace/merge_data_binary
        split: 'train' | 'val'
        """
        self.transform = transform
        self.samples = []
        
        # 'train'은 new_data, 'val'은 new_data_val 폴더를 사용
        data_dir_name = 'new_data' if split == 'train' else 'new_data_val'
        
        # 이미지 폴더 경로: data_root/new_data/img_binary
        img_base_dir = os.path.join(data_root, data_dir_name, 'img_binary')

        # 이진 감정 폴더 목록 (happy:1, other:0)
        self.classes = {'happy': 1, 'other': 0}

        print(f"--- {split} 데이터 로드 시작: {img_base_dir} ---")
        
        for class_name, label in self.classes.items():
            class_dir = os.path.join(img_base_dir, class_name)
            
            if not os.path.isdir(class_dir):
                print(f"[ERROR] 폴더를 찾을 수 없습니다: {class_dir}")
                continue

            for filename in os.listdir(class_dir):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.webp')):
                    img_path = os.path.join(class_dir, filename)
                    self.samples.append((img_path, label))

        # happy/other 카운트 확인
        happy_count = sum(1 for _, label in self.samples if label == 1)
        other_count = sum(1 for _, label in self.samples if label == 0)

        print(f"✅ {split} samples: {len(self.samples)} (Happy: {happy_count}, Other: {other_count})")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            # 파일 로드 실패 시 빈 이미지 반환 (실제 코드에서는 오류 처리 필요)
            print(f"이미지 로드 실패: {img_path}, {e}")
            return None, None 

        if self.transform is not None:
            image = self.transform(image)

        label = torch.tensor(label, dtype=torch.long)
        return image, label


# ================================
# 3. Transform / Dataset / DataLoader
# ================================
# ResNet 기본 입력 크기 224x224 기준
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# ★★★ 경로 수정 필요: merge_data_binary의 상위 폴더 경로 ★★★
# 예: /workspace/merge_data_binary가 데이터 폴더라면, 이 경로는 /workspace/ 이거나 /workspace/merge_data_binary 자체가 됩니다.
# 여기서는 data_root가 merge_data_binary 폴더를 가리킨다고 가정합니다.
IMG_BASE_ROOT = "/workspace/merge_data_binary" 


train_dataset = BinaryImageDataset(IMG_BASE_ROOT, split="train", transform=train_transform)
val_dataset   = BinaryImageDataset(IMG_BASE_ROOT, split="val",   transform=val_transform)

batch_size = 32
num_workers = 4

# DataLoader 생성 시, None 값을 필터링하기 위해 collate_fn을 정의합니다.
def collate_fn(batch):
    batch = list(filter(lambda x: x[0] is not None, batch))
    if not batch:
        return None, None 
    return torch.utils.data.dataloader.default_collate(batch)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=num_workers, collate_fn=collate_fn)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False,
                          num_workers=num_workers, collate_fn=collate_fn)


# ================================
# 4. 모델 정의 (ResNet18, 2-class)
# ================================
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)
model = model.to(device)

# 가중치 없는 기본 CrossEntropyLoss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)


# ================================
# 5. 학습 함수
# ================================
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        # ---- Train ----
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        for inputs, labels in tqdm(train_loader):
            if inputs is None: continue # collate_fn에서 None 반환 시 스킵

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)
        
        # total이 0인 경우를 방지
        if total == 0:
            print("Warning: No valid training samples found in this epoch.")
            continue 

        epoch_loss = running_loss / total
        epoch_acc = running_corrects.double() / total
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        val_total = 0

        with torch.inference_mode():
            for inputs, labels in val_loader:
                if inputs is None: continue # collate_fn에서 None 반환 시 스킵

                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
                val_total += labels.size(0)

        if val_total == 0:
            print("Warning: No valid validation samples found.")
            val_acc = 0.0
        else:
            val_loss /= val_total
            val_acc = val_corrects.double() / val_total
            print(f"Val   Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # best model 저장
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, "best_resnet_happy_vs_others.pth")
            print(">> best model updated")

    print(f"\nBest val Acc: {best_val_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model


# ================================
# 6. 학습 실행
# ================================
num_epochs = 10
model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs)


# ================================
# 7. 성능 지표 (val셋을 평가용으로 사용)
# ================================
model.eval()
all_labels = []
all_preds = []

with torch.inference_mode():
    for inputs, labels in val_loader:
        if inputs is None: continue
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# 예측 결과가 있을 경우에만 평가 지표 계산
if len(all_labels) > 0:
    acc  = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, pos_label=1)
    rec  = recall_score(all_labels, all_preds, pos_label=1)
    f1   = f1_score(all_labels, all_preds, pos_label=1)
    cm   = confusion_matrix(all_labels, all_preds)

    print("\n=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===")
    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1-score : {f1:.4f}")
    print("\nConfusion Matrix (rows: true, cols: pred)")
    print(cm)

    print("\nClassification report:")
    print(classification_report(all_labels, all_preds, target_names=["OTHER", "HAPPY"]))
else:
    print("\n🚨 평가할 데이터가 없어 성능 지표를 계산할 수 없습니다.")

device: cpu
--- train 데이터 로드 시작: /workspace/merge_data_binary/new_data/img_binary ---
✅ train samples: 9551 (Happy: 5203, Other: 4348)
--- val 데이터 로드 시작: /workspace/merge_data_binary/new_data_val/img_binary ---
✅ val samples: 1151 (Happy: 299, Other: 852)

Epoch 1/10
------------------------------


 85%|████████▍ | 254/299 [15:14<03:01,  4.03s/it]

이미지 로드 실패: /workspace/merge_data_binary/new_data/img_binary/happy/dc7e585097bcda58653ebf84fa33c6cc5d6f3eb500f105fa1f2693b1288fb632_남_20_기쁨_상업시설&점포&시장_20201209133321-003-023.jpg, broken data stream when reading image file


100%|██████████| 299/299 [17:47<00:00,  3.57s/it]

Train Loss: 0.2744 Acc: 0.8852


...

 58%|█████▊    | 173/299 [12:04<08:52,  4.23s/it]

이미지 로드 실패: /workspace/merge_data_binary/new_data/img_binary/happy/dc7e585097bcda58653ebf84fa33c6cc5d6f3eb500f105fa1f2693b1288fb632_남_20_기쁨_상업시설&점포&시장_20201209133321-003-023.jpg, broken data stream when reading image file


100%|██████████| 299/299 [20:35<00:00,  4.13s/it]

Train Loss: 0.0193 Acc: 0.9931


Val   Loss: 0.2447 Acc: 0.9374

Best val Acc: 0.9548

=== Val Metrics (binary: 1=HAPPY, 0=OTHERS) ===
Accuracy : 0.9548
Precision: 0.8946
Recall   : 0.9365
F1-score : 0.9150

Confusion Matrix (rows: true, cols: pred)
[[819  33]
 [ 19 280]]

Classification report:
              precision    recall  f1-score   support

       OTHER       0.98      0.96      0.97       852
       HAPPY       0.89      0.94      0.92       299

    accuracy                           0.95      1151
   macro avg       0.94      0.95      0.94      1151
weighted avg       0.96      0.95      0.96      1151

